# Домашняя работа
Прочитайте файлы в rdd, проведите их анализ, и расскажите, чем они отлачаются (для оценки отлично достаточно чтения файла и просмотра части его содержимого и сравнения названия колонок).




1.   [train файл](https://drive.google.com/file/d/1-ZUPzC1l-YqTpgT0M_R5Kmk54Iy0RMDA/view?usp=sharing)
2.   [test файл](https://drive.google.com/file/d/1-_1uEuafH0YR4C7Ksy2qcNj_0agi6SN_/view?usp=sharing)



## Описание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification

Скачиваем и копируем в папку с нотебуком, либо используем ссылки выше.

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

Все операции, доступные для rdd есть [тут](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations).


In [27]:
# Пример
from pyspark.sql import SparkSession


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.executor.instances', 4)\
        .config('spark.executor.memory', f'{int(3000/1.1)}mb')\
        .config('spark.executor.cores', 1)\
        .getOrCreate()

sc = spark.sparkContext

train_data = sc.textFile('train.csv')
test_data = sc.textFile('test.csv')

train_cols = train_data.take(1)[0].split(',')
test_cols = test_data.take(1)[0].split(',')
print('train data features num:', len(train_cols))
print('test data features num:', len(test_cols))
print('feats that are in train but not in test:', *[x for x in train_cols if x not in test_cols])
print('feats that are in test but not in train:', *[x for x in test_cols if x not in train_cols])


train data features num: 21
test data features num: 21
feats that are in train but not in test: price_range
feats that are in test but not in train: id


In [74]:
train_data.count()

2001

In [53]:
train_df = spark.read.options(header=True, inferSchema='True') \
    .csv('./train.csv')
test_df = spark.read.options(header=True, inferSchema='True') \
    .csv('./test.csv')

In [54]:
print('Train')
train_df.show(3)
print('Test')
test_df.show(3)

Train
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2